## Prototyping the construction of PySAL weights from geojson. Supports local files, uri, and serialized objects 

In [1]:
import geojson
import pysal as ps
import urllib2 as urllib
import numpy as np

In [2]:
class PolygonCollection:
    def __init__(self, polygons, bbox=None):
        """

        Parameters
        ==========
        polygons: dict
                  key is polygon Id, value is PySAL Polygon object
        bbox: list (optional)
              [left, lower, right, upper]

        Notes
        =====
        bbox is supported in geojson specification at both the feature and feature collection level. However, not all geojson writers generate the bbox at the feature collection level. 
        In those cases, the bbox property will be set on initial access.

        """
              
        self.type=ps.cg.Polygon
        self.n = len(polygons)
        self.polygons = polygons
        self.ids = polygons.keys()
        
        if bbox is None:
            self._bbox = None
        else:
            self._bbox = bbox
            
    @property
    def bbox(self):
        bboxes = np.array([self.polygons[p].bbox for p in self.polygons])
        mins = bboxes.min(axis=0)
        maxs = bboxes.max(axis=0)
        self._bbox = [ mins[0], mins[1], maxs[2], maxs[3] ]
        return self._bbox
        
    
    def __getitem__(self, index):
        return self.polygons[index]
            

In [3]:
def queen_geojson(gjobj):
    """
    Constructs a PySAL queen contiguity W from a geojson object

    """
    polys = []
    ids = []
    i = 0
    for feature in gjobj['features']:
        polys.append(ps.cg.asShape(geojson.Polygon(feature['geometry']['coordinates'])))
        ids.append(i)
        i += 1
    polygons = PolygonCollection(dict(zip(ids,polys)))
    neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(polygons).w
    return ps.W(neighbors)

In [4]:
def queen_from_geojson(uri):
    gjobj = geojson.load(urllib.urlopen(uri))
    return queen_geojson(gjobj)

In [5]:
def queen_from_geojsons(s):
    return queen_geojson(info)

In [6]:
def queen_from_geojsonf(fileName):
    fp = open(fileName)
    obj = geojson.load(fp)
    fp.close()
    return queen_geojson(obj)

In [7]:
# from a uri
uri = "http://toae.org/pub/columbus.json"

In [8]:
wq = queen_from_geojson(uri)

In [9]:
wq.n

49

In [10]:
# from a local file
wf = queen_from_geojsonf(ps.examples.get_path("columbus.json"))

In [11]:
wf.n

49

In [12]:
# from a serialized geojson object

fp = open(ps.examples.get_path("columbus.json"))
info = geojson.load(fp)
fp.close()
infos = geojson.dumps(info)
ws = queen_from_geojsons(infos)

In [13]:
ws.n

49

In [14]:
wf.neighbors == ws.neighbors

True

In [15]:
wq.neighbors == wf.neighbors

True

## Prototyping the construction of PySAL weights from shapely polygons.

The following function returns the weights object created based on contiguity criterion. Needs to be extended to other types. Doesnt handle errors like geoObj containing lines, points for now.

In [16]:
import pysal as ps
from pysal.weights._contW_binning import ContiguityWeightsPolygons
from pysal.cg.shapes import asShape

In [17]:
WT_TYPE = {'queen': 1, 'rook': 2}

In [22]:
def weights_from_geoObject(geoObj, ids, wType = 'queen'):
    '''
    Returns a weights object.
    
    Parameters:
    ==========
    
    geoObj: List 
            of polygons supporting the __geo_interface__.
    
    ids: List
            of ids for the polygons.
    
    wType: 'queen' or 'rook'
        Type of contiguity.
    
    '''
    polys = map(asShape, geoObj)
    
    polys = dict(zip(ids, polys))
    
    polys = PolygonCollection(polys)
    
    neighbours = ContiguityWeightsPolygons(polys, WT_TYPE[wType]).w
    
    return ps.W(neighbours)

In [23]:
from shapely.geometry import Polygon

In [24]:
bl = [(x, y) for y in range(3) for x in range(3)]
br = [(x, y) for y in range(3) for x in range(1,4)]
tr = [(x, y) for y in range(1,4) for x in range(1,4)]
tl = [(x,y) for y in range(1,4) for x in range(3)]

polys = map(Polygon, zip(bl,br,tr,tl))

w = weights_from_geoObject(polys, range(1, 10))

wfull = w.full()
print wfull


(array([[ 0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
       [ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.]]), [0, 1, 2, 3, 4, 5, 6, 7, 8])
